In [7]:
import numpy as np
import cPickle as pickle
import gzip

----

In [298]:
class LinearLayer(object):
    def __init__(self, input_dim, output_dim):
        self.W = np.random.randn(output_dim, input_dim)
    def forward_propagation(self, input_):
        return self.W.dot(input_)
    def backward_propagation(self, input_, gradient_wrt_output):
        pass
    def update_parameters(self, input_, gradient_wrt_output, learning_rate):
        pass

In [386]:
class SigmoidLayer(object):
    def __init__(self):
        pass
    def forward_propagation(self, input_):
        #return np.apply_along_axis(lambda x : 1 / (1 + np.exp(-x)), axis=1, arr=input_)
        return 1 / (1 + np.exp(-input_))
    def backward_propagation(self, input_, gradient_wrt_output):
        pass
    def update_parameters(self, input_, gradient_wrt_output, learning_rate):
        pass

In [385]:
class SoftmaxLayer(object):
    def __init__(self):
        pass
    def forward_propagation(self, input_):
        #return np.apply_along_axis( lambda x: np.exp(x) / np.sum(np.exp(x)), axis=1, arr=input_)
        return np.exp(input_) / np.sum(np.exp(input_))
    def backward_propagation(self, input_, gradient_wrt_output):
        raise NotImplementedException()

In [268]:
def one_hot(indices, num_classes):
    b = np.zeros( (indices.shape[0], num_classes) )
    for i in range(0, b.shape[0]):
        b[i, indices[i]-1 ] = 1
    return b

----

In [9]:
with gzip.open("../data/mnist.pkl.gz") as f:
    train_set, valid_set, test_set = pickle.load(f)

In [489]:
Xt, yt = train_set

In [16]:
Xt.shape, yt.shape

((50000, 784), (50000,))

$X \in R^{b \times p}$, where $b$ is the batch size and $p$ is the number of attributes

In [316]:
Xt_batch = Xt[0:1]
yt_batch = one_hot( yt[0:1], num_classes=10)
print Xt_batch.shape
print yt_batch.shape

(1, 784)
(1, 10)


In [321]:
layers = [LinearLayer(784, 100), SigmoidLayer(), LinearLayer(100, 10), SoftmaxLayer()]

In [322]:
inputs = [Xt_batch.T]
print "output shape: %s" % str(inputs[0].shape)
for layer in layers:
    inputs.append( layer.forward_propagation(inputs[-1]))
    print "output shape: %s" % (str(inputs[-1].shape))

output shape: (784, 1)
output shape: (100, 1)
output shape: (100, 1)
output shape: (10, 1)
output shape: (10, 1)


In [323]:
activations = inputs[-1]

In [324]:
activations

array([[  1.18884540e-07],
       [  2.57296012e-04],
       [  4.18649281e-03],
       [  1.07564875e-05],
       [  9.95066677e-01],
       [  4.64094912e-04],
       [  1.37372161e-05],
       [  4.78145991e-08],
       [  7.78695972e-07],
       [  1.07216160e-10]])

In [325]:
for row in inputs[-1]:
    assert sum(row) - 1.0 < 1e-6

Let us now define the loss function

In [340]:
def loss(input_, targets):
    targets = targets.T
    assert input_.shape[0] == targets.shape[0]
    return -np.log(np.sum(targets * activations))

In [341]:
loss(activations, yt_batch)

0.0049455319476775228

----

In [345]:
y = activations
t = yt_batch.T

In [346]:
print y.shape
print t.shape

(10, 1)
(10, 1)


We want to compute $\frac{dL}{dy}$, where $y = softmax(y')$. This is what the softmax layer will propagate back to the linear layer before it.

In [350]:
dL_dy = -t / y
print dL_dy.shape

(10, 1)


Compute $\frac{dL}{dy'}$, where $y' = Vh + c$. We will be backpropagating this to the previous layer (the sigmoid). Also, while we're inside this layer, we should also compute the derivatives needed for the parameter updates: $\frac{dL}{dV}$ and $\frac{dL}{dc}$ (these aren't backpropagated).

In [351]:
dL_dyprime = y - t
print dL_dyprime.shape

(10, 1)


In [367]:
h = inputs[2] # h = inputs[2] = sigm(Wx+b)
dL_dV = np.dot(dL_dyprime, h.T)
print dL_dV.shape

(10, 100)


In [368]:
dL_dc = dL_dyprime

Now let us compute $\frac{dL}{dh}$, where $h = sigmoid(h') = sigmoid(Wx + b)$. We will be backpropagating $\frac{dL}{dh}$ to the previous (and beginning) layer.

In [369]:
V = layers[2].W
V.shape

(10, 100)

In [370]:
dL_dh = np.dot(V.T, dL_dyprime)
dL_dh.shape

(100, 1)

Okay, last time we do this. We have $\frac{dL}{dh}$ from the following layer and all we need to do is compute $\frac{dL}{dh'}$. From this, we can compute the parameter gradients $\frac{dL}{dW}$ and $\frac{dL}{db}$ and we'll be done.

In [371]:
h_prime = inputs[1] # h' = inputs[1] = Wx+b

In [375]:
# dL_dh = dL_dh * sigmoid(h') * (1 - sigmoid(h'))
dL_dhprime = dL_dh * h * (1 - h)
dL_dhprime.shape

(100, 1)

In [378]:
dL_dW = np.dot(dL_dhprime, inputs[0].T)
dL_dW.shape

(100, 784)

In [379]:
dL_db = dL_dhprime
dL_db.shape

(100, 1)

----

Ok, let's try this by implementing backward_propagation for each layer

In [408]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

In [519]:
class OneLayerMLP(object):
    def __init__(self, num_input_units, num_hidden_units, num_output_units):
        self.W = np.random.randn(num_hidden_units, num_input_units)
        self.V = np.random.randn(num_output_units, num_hidden_units)
        self.b = np.random.randn(num_hidden_units, 1)
        self.c = np.random.randn(num_output_units, 1)
        self.num_input_units = num_input_units
        self.num_hidden_units = num_hidden_units
        self.num_output_units = num_output_units
    def loss(self, activation, target):
        assert activation.shape[0] == target.shape[0]
        return -np.log(np.sum(target * activation))
    def classify(self, x):
        return self.forward(x)[-1]
    def forward(self, x):
        assert x.shape == (self.num_input_units, 1)
        hprime = np.dot(self.W, x) + self.b
        h = sigmoid(hprime)
        yprime = np.dot(self.V, h) + self.c
        y = softmax(yprime)
        assert y.shape == (self.num_output_units, 1)
        return [x, hprime, h, yprime, h, y]
    def backprop(self, inputs, target, learning_rate):
        x = inputs[0]
        y = inputs[-1]
        dL_dy = -t / y
        dL_dyprime = y - t
        h = inputs[2] # h = inputs[2] = sigm(Wx+b)
        dL_dV = np.dot(dL_dyprime, h.T)
        dL_dc = dL_dyprime
        dL_dh = np.dot(self.V.T, dL_dyprime)
        h_prime = inputs[1]
        dL_dhprime = dL_dh * h * (1 - h)
        dL_dW = np.dot(dL_dhprime, x.T)
        dL_db = dL_dhprime
        # test
        assert dL_dW.shape == self.W.shape
        assert dL_db.shape == self.b.shape
        assert dL_dV.shape == self.V.shape
        assert dL_dc.shape == self.c.shape
        # ok do the parameter updates
        self.W = self.W - learning_rate*dL_dW
        self.b = self.b - learning_rate*dL_db
        self.V = self.V - learning_rate*dL_dV
        self.c = self.c - learning_rate*dL_dc

In [508]:
np.random.seed(1)
mlp = OneLayerMLP(num_input_units=784, num_hidden_units=100, num_output_units=10)

In [520]:
for epoch in range(0, 10):
    this_losses = []
    for i in range(0, Xt.shape[0]):
        Xt_batch = Xt[i:i+1].T
        yt_batch = one_hot( yt[i:i+1], num_classes=10 ).T
        outputs = mlp.forward(Xt_batch)
        this_losses.append(mlp.loss(outputs[-1], yt_batch))
        mlp.backprop(outputs, yt_batch, 0.01)
    print "this loss: %d" % (np.mean(this_losses))

KeyboardInterrupt: 

In [513]:
yt[0:10].T

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4])

In [515]:
np.argmax(mlp.classify(Xt[2:2+1].T))

4

In [483]:
outs = mlp.forward(Xt_batch.T, yt_batch.T)
outs[-1]

array([[  6.22259175e-10],
       [  1.12633509e-07],
       [  3.88029628e-08],
       [  7.61745343e-13],
       [  2.76263221e-06],
       [  5.38590484e-11],
       [  9.04481362e-10],
       [  1.54028889e-01],
       [  1.67873714e-04],
       [  8.45800322e-01]])

In [479]:
mlp.backprop(outs, yt_batch, 0.1)
mlp.c

array([[ 0.18877156],
       [ 1.47475715],
       [ 2.04789582],
       [ 0.07034556],
       [-0.17854169],
       [ 1.01649925],
       [-1.09767397],
       [-1.05564543],
       [ 2.73785476],
       [ 2.18934815]])